In [1]:
from mpi4py import MPI
from manapy.ast import Variable
from manapy.base.base import Struct
from manapy.ddm import Domain
from manapy.partitions import MeshPartition
from manapy.solvers.advec.tools_utils import initialisation_gaussian_2d
import numpy as np
import time
from numba import cuda

import matplotlib.pyplot as plt

from timeit import default_timer as timer
from manapy.cuda.utils import (VarClass, GPU_Backend)
###############
# test_time
###############

def test_time(iter, fun):
  #fun()
  start_time = timer()
  for _ in range(iter):
    fun()
  end_time = timer()
  elapsed_time = (end_time - start_time) / iter
  print(f"{elapsed_time * 1000:.5f} ms")
  #print(f"{elapsed_time * 1000000:.5f} micros")

###############
# init
###############
def init(dim, mesh_path):
  GPU_Backend.float_precision = 'float32'
  running_conf = Struct(backend="numba", signature=True, cache=True, float_precision="single")
  MeshPartition(mesh_path, dim=dim, conf=running_conf, periodic=[0,0,0])

  domain = Domain(dim=dim, conf=running_conf)
  ne = Variable(domain=domain)
  u  = Variable(domain=domain)
  v  = Variable(domain=domain)
  w  = Variable(domain=domain)
  
  P = Variable(domain=domain)
  Pinit = 2.0
  cells = domain.cells
  initialisation_gaussian_2d(ne.cell, u.cell, v.cell, P.cell, cells.center, Pinit)

  u.face[:] = 2.
  v.face[:] = 0.
  w.face[:] = 0.
  
  u.interpolate_facetocell()
  v.interpolate_facetocell()
  w.interpolate_facetocell()
  return (domain, ne, u, v, w, P, running_conf)

In [2]:
from manapy.solvers.advec import AdvectionSolver

dim = 2
#mesh_file = "/home/aben-ham/Desktop/work/stage/my_manapy/manapy/mesh/2D/carre.msh"
#mesh_file = "/home/ayoub.hamou/mesh/square.msh"
mesh_file = "/home/aben-ham/Desktop/work/stage/my_manapy/gpu_accelerator/functions/square.msh"
domain, ne, u, v, w, P, conf = init(dim=dim, mesh_path=mesh_file)

S = AdvectionSolver(ne, vel=(u, v), conf=conf)

Reading gmsh file ...
Saving partition files ...
Number of Cells: 258030
Number of Vertices: 129684
Local domain contruction ...


In [4]:
# args list
#? The order is imported of the argument list

S.u.cell[:] = 1
S.v.cell[:] = 1
S.w.cell[:] = 1
S.domain.faces.normal[:] = 1

VarClass.convert_to_var_class([
    domain.nodes,
    domain.faces,
    domain.cells,
    domain.halos,
    domain,
    ne,
    S,
    u,
    v,
    S.w,
    ne.domain._cells,

])

args = [
  S.u.cell,
  S.v.cell,
  S.w.cell,
  np.float32(S.cfl),
  S.domain.faces.normal,
  S.domain.faces.mesure,
  S.domain.cells.volume,
  S.domain.cells.faceid,
  S.dim,
]

for arg in args:
  print(type(arg))

---------------------------------
---------------------------------
can't get attr _nbnodes => _nbnodes
can't get attr nbnodes => _nbnodes
can't set attr for R_x => can't set attribute
can't set attr for R_y => can't set attribute
can't set attr for R_z => can't set attribute
set att: _R_x
set att: _R_y
set att: _R_z
set att: _cellid
set att: _ghostcenter
set att: _ghostfaceinfo
set att: _ghostid
set att: _haloghostcenter
set att: _haloghostfaceinfo
set att: _haloghostid
set att: _halonid
set att: _lambda_x
set att: _lambda_y
set att: _lambda_z
set att: _loctoglob
set att: _name
set att: _number
set att: _oldname
set att: _periodicid
set att: _vertex
can't set attr for cellid => can't set attribute
can't set attr for ghostcenter => can't set attribute
can't set attr for ghostfaceinfo => can't set attribute
can't set attr for ghostid => can't set attribute
can't set attr for haloghostcenter => can't set attribute
can't set attr for haloghostfaceinfo => can't set attribute
can't set attr

In [5]:
from manapy.solvers.advec.fvm_utils import time_step as cpu_function
from manapy.cuda.manapy.solvers.advec.cuda_fvm_utils import get_kernel_time_step as gpu_function

numba_fun = domain.backend.compile(cpu_function, echo=True, signature=True)
cuda_fun = gpu_function()

compile time_step to cpu => signature=(float32[:], float32[:], float32[:], float32, float32[:,:], float32[:], float32[:], int32[:,:], int32)
compile kernel_time_step to cuda => signature=void(float32[:], float32[:], float32[:], float32, float32[:,:], float32[:], float32[:], int32[:,:], int32, float32[:])
compile kernel_assign to cuda => signature=void(float32[:], float32)


In [7]:
numba_fun(*args)
%timeit numba_fun(*args)

17.5 ms ± 3.06 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
cuda_fun(*args)
%timeit cuda_fun(*args)

CUDADispatcher(<function get_kernel_time_step.<locals>.kernel_time_step at 0x7de4e3506820>) is called
VarClass<float32, (258030,)>
VarClass<float32, (258030,)>
VarClass<float32, (258030,)>
<class 'numpy.float32'>
VarClass<float32, (387713, 3)>
VarClass<float32, (387713,)>
VarClass<float32, (258030,)>
VarClass<int32, (258030, 4)>
<class 'numpy.int32'>


/home/aben-ham/anaconda3/envs/stage/lib/python3.8/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/home/aben-ham/anaconda3/envs/stage/lib/python3.8/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 32 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


2.23 ms ± 42.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
cpu_res = cuda_fun(*args)
gpu_res = cuda_fun(*args)
print(cpu_res, gpu_res, abs(cpu_res - gpu_res))

1.0363967e-07 1.0363967e-07 0.0
